IMPORTAÇÃO DE MODULOS

In [145]:
from path_arquivos import *
import pandas as pd
import numpy as np

LEITURA DOS DATAFRAMES

In [146]:
df_prod = pd.read_excel(ar_xlsx.ar_96, usecols= ['CODPROD', 'QTUNITCX', 'CAPACIDADE', 'PONTOREPOSICAO','QTTOTPAL'])
df_bloq = pd.read_excel(ar_xls.ar_86, usecols=['Código', 'Bloqueado(Qt.Bloq.-Qt.Avaria)'])
end_ger = pd.read_csv(ar_csv.ar_07, header= None, names= col_name.c07)  
df_end = pd.read_csv(ar_csv.ar_end, header= None, names= col_name.cEnd)



c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


CORPO DO SCRIPT

In [147]:
dic_end_ger = {'COD' : "CODPROD"}
end_ger = end_ger.rename(columns= dic_end_ger)
end_ger['RUA'] = end_ger['RUA'].fillna(0).astype(int)

dic_bloq = {'Código' : "CODPROD", 'Bloqueado(Qt.Bloq.-Qt.Avaria)' : "BLOQ"}
df_bloq = df_bloq.rename(columns= dic_bloq)
df_bloq['CODPROD'] = df_bloq['CODPROD'].fillna(0).astype(int)

dic_end = {"COD" : "CODPROD"}
df_end = df_end.rename(columns= dic_end)
df_end = df_end[['CODPROD','ENTRADA', 'SAIDA', 'DISP','QTDE']]

df_prod['CODPROD'] = df_prod['CODPROD'].fillna(0).astype(int)

df = end_ger.loc[end_ger['RUA'].between(1, 39)]
df = df.merge(df_bloq, left_on='CODPROD',right_on='CODPROD', how= 'left')
df = df.merge(df_end, left_on='CODPROD',right_on='CODPROD', how= 'left')
df = df.merge(df_prod, left_on='CODPROD',right_on='CODPROD', how= 'left')

drop_col = ['EMBALAGEM', 'CAPACIDADE','PONTOREPOSICAO']
df.drop(columns= drop_col, inplace= True)

VISUALIZAÇÃO

In [148]:

display(df_prod)
display(df_end)
display(df_bloq)
display(end_ger)
display(df)

,CODPROD,QTUNITCX,QTTOTPAL,CAPACIDADE,PONTOREPOSICAO
0,425617,1,4,10000,5000
1,433306,1,4,10000,5000
2,422849,1,4,10000,5000
3,429956,1,4,10000,5000
4,432208,1,4,10000,5000
...,...,...,...,...,...
14205,475112,24,234,10,2
14206,475090,6,174,10,2
14207,475101,24,234,10,2
14208,475111,24,234,10,2


,CODPROD,ENTRADA,SAIDA,DISP,QTDE
0,472097,0,0,384,384
1,465661,0,0,40,40
2,61201,0,0,60,60
3,461396,0,0,256,256
4,61301,0,0,91,91
...,...,...,...,...,...
22625,466638,0,0,132,132
22626,466635,0,0,1200,1200
22627,470741,0,0,144,144
22628,466634,0,0,1500,1500


,CODPROD,BLOQ
0,474720,0.0
1,443825,0.0
2,444079,0.0
3,444066,0.0
4,433054,0.0
...,...,...
42542,437101,0.0
42543,438953,0.0
42544,432964,0.0
42545,464176,0.0


,CODPROD,DESCRICAO,DEP,RUA,PREDIO,NIVEL,APTO,EMBALAGEM,CAP,P_REP,QTDE_O.S,PICKING,PULMAO,ENDERECO,GERENCIAL
0,2001,ABS NOVA NORMAL GEL C/ABAS C/10,NaN,0,NaN,NaN,NaN,CX/0048/UN,NaN,NaN,0,0,0,"0,00","0,00"
1,3101,SACHET ADOC ZERO CAL 8GR PO ASPART 50UN,NaN,0,NaN,NaN,NaN,CX/0012/UN,NaN,NaN,0,0,0,"0,00","0,00"
2,3201,ADOC ZERO CAL CICLAMATO 100ML,1.0,3,53.0,1.0,101.0,CX/0012/UN,396.0,49.0,2,268,0,"268,00","269,00"
3,3301,ADOC ZERO CAL CICLAMATO 200ML,1.0,3,84.0,1.0,103.0,CX/0012/UN,60.0,18.0,0,45,169,"214,00","214,00"
4,3401,AGUA OXIG CREM FARMAX 20 V 90ML,NaN,0,NaN,NaN,NaN,PT/0024/UN,NaN,NaN,0,0,0,"0,00","0,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43618,475980,AMAC BRILUX 500ML CONC FLORES DE PRIMAVE,NaN,0,NaN,NaN,NaN,CX/0012/UN,NaN,NaN,0,0,0,"0,00","0,00"
43619,475981,AMAC BRILUX 500ML CONC JARDIM DE INVERNO,NaN,0,NaN,NaN,NaN,UN/0012/UN,NaN,NaN,0,0,0,"0,00","0,00"
43620,475982,AMAC BRILUX 500ML CONC JARDIM DE INVERNO,NaN,0,NaN,NaN,NaN,UN/0012/UN,NaN,NaN,0,0,0,"0,00","0,00"
43621,475983,SAB EVEN 85GR FLORES E BURITI,NaN,0,NaN,NaN,NaN,PT/0012/UN,NaN,NaN,0,0,0,"0,00","0,00"


,CODPROD,DESCRICAO,DEP,RUA,PREDIO,NIVEL,APTO,CAP,P_REP,QTDE_O.S,...,PULMAO,ENDERECO,GERENCIAL,BLOQ,ENTRADA,SAIDA,DISP,QTDE,QTUNITCX,QTTOTPAL
0,3201,ADOC ZERO CAL CICLAMATO 100ML,1.0,3,53.0,1.0,101.0,396.0,49.0,2,...,0,"268,00","269,00",0.0,0.0,0.0,248,248,1,352
1,3301,ADOC ZERO CAL CICLAMATO 200ML,1.0,3,84.0,1.0,103.0,60.0,18.0,0,...,169,"214,00","214,00",1.0,0.0,0.0,132,132,1,168
2,3301,ADOC ZERO CAL CICLAMATO 200ML,1.0,3,84.0,1.0,103.0,60.0,18.0,0,...,169,"214,00","214,00",1.0,0.0,0.0,36,36,1,168
3,3601,AGUA SANITARIA Q-BOA 1LT,1.0,22,13.0,1.0,101.0,84.0,14.0,2,...,841,"849,00","852,00",3.0,0.0,0.0,70,70,1,70
4,3601,AGUA SANITARIA Q-BOA 1LT,1.0,22,13.0,1.0,101.0,84.0,14.0,2,...,841,"849,00","852,00",3.0,0.0,0.0,70,70,1,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24701,475920,"SABAO PO MINUANO 2,2KG MAX LIMPEZA AZ",1.0,14,63.0,1.0,101.0,48.0,8.0,0,...,440,"480,00","480,00",0.0,0.0,0.0,40,40,1,40
24702,475920,"SABAO PO MINUANO 2,2KG MAX LIMPEZA AZ",1.0,14,63.0,1.0,101.0,48.0,8.0,0,...,440,"480,00","480,00",0.0,0.0,0.0,40,40,1,40
24703,475920,"SABAO PO MINUANO 2,2KG MAX LIMPEZA AZ",1.0,14,63.0,1.0,101.0,48.0,8.0,0,...,440,"480,00","480,00",0.0,40.0,0.0,40,0,1,40
24704,475920,"SABAO PO MINUANO 2,2KG MAX LIMPEZA AZ",1.0,14,63.0,1.0,101.0,48.0,8.0,0,...,440,"480,00","480,00",0.0,0.0,0.0,40,40,1,40
